In this example, we will clean, merge, and pivot data from eurostat. 

The data in question is two separate files regarding historical flight data between European airports and European countries. 

The first file is for passenger flight data
The second file is for freight flight data

We want to merge them to get a full picture of flight data in Europe over time. 

Data is found here: 
https://ec.europa.eu/eurostat/data/database?p_p_id=NavTreeportletprod_WAR_NavTreeportletprod_INSTANCE_nPqeVbPXRmWQ&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-2&p_p_col_pos=1&p_p_col_count=2

In [1]:
import pandas as pd
import re
import os

In [2]:
data_file = os.path.join("..")

In [3]:
first_file = os.path.join(data_file,"avia_paoac.tsv")
second_file = os.path.join(data_file,"avia_gooac.tsv")

In [4]:
#Definitions we will need to translate Eurostat codes to meaningful variables
airports = pd.read_csv("../eurostat_dicts/rep_airp.csv")
measures = pd.read_csv("../eurostat_dicts/tra_meas.csv")
countries = pd.read_csv("../eurostat_dicts/CountryCodes.csv")

In [5]:
def import_tsv(tsvfile):
    """Imports a tsv (tab separated file), which is how Eurostat exports their data"""
    df = pd.read_csv(tsvfile,sep = '\t')
    return df

In [6]:
def get_value(string):
    ''' Converts values to floats. Accounts for the many formats Eurostat uses like ":" and "0 d"'''
    if isinstance(string,int):
        line = string
    else:
        line = re.sub('[A-Za-z:]','',string).strip()
        try:
            line = float(line)
        except:
            if line == '':
                line = 0
            else:
                print(string)
    return line

In [7]:
def split_cols(df):
    """Converts the first column into it's different constitutent parts. This will work for all Eurostat files."""
    counter = 0
    for i in df.columns[0].split("\\")[0].split(","):
        df[i] = [x.split(",")[counter] for x in df.iloc[:,0]]
        counter +=1
    return df

In [8]:
def get_months(df):
    """This function is specifally for data that has monthly, quarterly, and annual data in the same file. It will only
    take the monthly data."""
    monthly_cols = []
    for c in df.columns:
        if "M" in c:
            monthly_cols.append(c)
    monthly_cols.append('Country_x')
    monthly_cols.append("Label_y")
    monthly_cols.append("Label_x")
    monthly_cols.append('Country_y') 
    return df[monthly_cols]

In [9]:
def all_together(tsv_file):
    """Converts the air transportation files to a pivoted, readable format."""
    df = import_tsv(tsv_file)
    df = split_cols(df)
    #One extra split so we can get the Country for each airport
    df['origin_country']= [x.split("_")[0] for x in df['rep_airp']]
    #Merge with definition files
    df = df.merge(airports[['Notation','Label']],left_on = 'rep_airp',right_on = 'Notation',how = 'left')
    df = df.merge(measures[['Notation','Label']],left_on = 'tra_meas',right_on = 'Notation',how = 'left')
    df = df.merge(countries,left_on = 'partner',right_on = 'Code',how = 'left')
    df = df.merge(countries,left_on = 'origin_country',right_on = 'Code',how = 'left')
    #Get just monthly data
    filtered = get_months(df)
    #Melt down to turn MonthYear into one date column
    melt = filtered.melt(id_vars = ['Country_x','Country_y','Label_y','Label_x'],value_vars = filtered.columns[0:-4])
    #Make sure the values are all floats
    melt['Value']= melt['value'].apply(get_value)
    #Take only the columns we care about
    out = melt[['Country_x','Country_y','Label_y','Label_x','variable','Value']]
    #Rename columns
    out.columns = ['Destination Country','Origin Country','Variable','Airport','YearMonth','Value']
    #Pivot this data so that each variable has its own column
    piv = out.pivot_table(index = ['Destination Country','Origin Country','Airport','YearMonth'],columns = 'Variable')
    piv.columns = piv.columns.get_level_values(1)
    piv = piv.reset_index()
    return piv

In [10]:
out1 = all_together(first_file)

In [11]:
out2 = all_together(second_file)

In [12]:
together = out1.merge(out2,on = ['Destination Country','Origin Country','Airport','YearMonth'],how = 'left')

In [13]:
together.head()

Variable,Destination Country,Origin Country,Airport,YearMonth,Commercial passenger air flights,Commercial passenger air flights (arrivals),Commercial passenger air flights (departures),Passengers carried,Passengers carried (arrival),Passengers carried (departures),...,Passengers on board (departures),Freight and mail commercial air flights,Freight and mail commercial air flights (arrivals),Freight and mail commercial air flights (departures),Freight and mail loaded,Freight and mail loaded and unloaded,Freight and mail on board,Freight and mail on board (arrivals),Freight and mail on board (departures),Freight and mail unloaded
0,Austria,Austria,GRAZ airport,1993M01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Austria,Austria,GRAZ airport,1993M02,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Austria,Austria,GRAZ airport,1993M03,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Austria,Austria,GRAZ airport,1993M04,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Austria,Austria,GRAZ airport,1993M05,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
together = together.fillna(0)

In [15]:
together.columns

Index(['Destination Country', 'Origin Country', 'Airport', 'YearMonth',
       'Commercial passenger air flights',
       'Commercial passenger air flights (arrivals)',
       'Commercial passenger air flights (departures)', 'Passengers carried',
       'Passengers carried (arrival)', 'Passengers carried (departures)',
       'Passengers on board', 'Passengers on board (arrivals)',
       'Passengers on board (departures)',
       'Freight and mail commercial air flights',
       'Freight and mail commercial air flights (arrivals)',
       'Freight and mail commercial air flights (departures)',
       'Freight and mail loaded', 'Freight and mail loaded and unloaded',
       'Freight and mail on board', 'Freight and mail on board (arrivals)',
       'Freight and mail on board (departures)', 'Freight and mail unloaded'],
      dtype='object', name='Variable')

In [16]:
together['Total Flights']= together['Commercial passenger air flights']+together['Freight and mail commercial air flights']

In [17]:
together.to_csv("../Datasets/Flight_Pass&Freight_EU_93-17.csv",index = False)